# Airbus Hackatuna

**Table of contents** :

- 1°/ Initialization :

# 1°/ Initialization :

In [1]:
import pandas as pd
import numpy as np

import tqdm

import os
import sys
sys.path.append('../')
sys.path.append('../src')

import config as c
import utils as u
import lidar_utils as lu

In [2]:
landscape_dfs = u.first_preprocessing()

In [28]:
def subsample_frame(frame_df, label, n_points = 4096):
    
    frame_df = frame_df[frame_df["label"] != "33"]

    # encode labels
    labels = frame_df["label"]
    points = frame_df[["x", "y", "z", "reflectivity"]].values

    unique_classes = np.unique(labels)
    per_class = n_points // len(unique_classes)

    idxs = []

    for c in unique_classes:
        cls_idx = np.where(labels == c)[0]
        if len(cls_idx) <= per_class:
            idxs.extend(cls_idx.tolist())
        else:
            idxs.extend(
                np.random.choice(cls_idx, per_class, replace=False)
            )

    # pad if needed
    if len(idxs) < n_points:
        extra = np.random.choice(
            idxs, n_points - len(idxs), replace=True
        )
        idxs.extend(extra.tolist())

    idxs = np.array(idxs)
    return points[idxs], labels[idxs]

In [21]:
import h5py

def write_split(h5_path, landscape_dfs, landscape_ids, label, n_points = 4096):
    with h5py.File(h5_path, "w") as h5f:
        for lid in landscape_ids:
            df = landscape_dfs[lid]
            lg = h5f.create_group(f"landscape_{lid}")

            for pose_idx, frame_df in df.groupby("pose_index"):
                pts, lbls = subsample_frame(frame_df, label, n_points)

                fg = lg.create_group(f"frame_{pose_idx:04d}")

                fg.create_dataset(
                    "points",
                    data=pts.astype(np.float32),
                    compression="gzip"
                )
                fg.create_dataset(
                    "labels",
                    data=lbls.astype(np.uint8),
                    compression="gzip"
                )

In [27]:
#write_split(
#    "datasets/processed/train.h5",
#    landscape_dfs,
#    c.TRAIN_LS,
#²    n_points = 4096
#)
#
#write_split(
#    "datasets/processed/val.h5",
#    landscape_dfs,
#    c.VAL_LS,
#    n_points = 4096
#)

write_split(
    "../datasets/processed/test.h5",
    landscape_dfs,
    [8, 9],
    {
    "Antenna": 0,
    "Cable": 1,
    "Electric pole": 2,
    "Wind turbine": 3,
    "Other": 33
},
    n_points = 4096
)

KeyError: "None of [Index([ 12969, 127139, 102444, 150410, 154711,  64871,  75127,  67767,  17357,\n        44737,\n       ...\n        39746,  93908,  44983, 119610,   8829,   4717,  34810,   5488, 137739,\n        26401],\n      dtype='int64', length=4096)] are in the [index]"

In [6]:
labels = landscape_dfs[1]["label"].map(c.LABEL_MAP)
unique_classes = landscape_dfs[1]["label"].unique()
unique_classes

array([33,  3,  1,  0,  2], dtype=int32)

In [19]:
for lid in [8, 9]:
    print(lid)

8
9
